In [1]:
import os
from os.path import isfile, join
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
#from scipy.ndimage import rotate
import cv2 
import json
from huggingface_hub import cached_download, hf_hub_url
from transformers import AutoModelForSemanticSegmentation
from transformers import AutoFeatureExtractor
from Video_processing.video_image_utils import *
from palette import *

/Users/niche/miniconda3/envs/mymconda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
### Load pretraint model to initialize parameters

repo_id = "huggingface/label-files"
#filename = "ade20k-hf-doc-builder.json" ### this file name was written in the tutorial, but it is not working
filename= "ade20k-id2label.json"
id2label = json.load(open(cached_download(hf_hub_url(repo_id, filename, repo_type="dataset")), "r"))
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)

pretrained_model_name = "nvidia/mit-b0"
model = AutoModelForSemanticSegmentation.from_pretrained(
    pretrained_model_name, id2label=id2label, label2id=label2id
)

/Users/niche/miniconda3/envs/mymconda/lib/python3.9/site-packages/huggingface_hub/file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(
Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model c

In [6]:
### feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained("nvidia/mit-b0", reduce_labels=True)

In [7]:
### loding finetuned model
model.load_state_dict(torch.load('/Users/niche/Transformer_study/Trained_Model_with_50_images/pytorch_model.bin'))

<All keys matched successfully>

In [5]:
#''' 
### loading an Image from Directory
### 
try:
    if not os.path.exists('seg_images'):
        os.makedirs('seg_images')
except OSError:
    print ('Error: Creating directory of seg_images')

pathIn='./Video_processing/Test_images/' 
pathOut_seg='./seg_images/'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # use GPU if available, otherwise use a CPU
#pathIn='./test_image/study_room.jpeg/' ## cow
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
files. sort(key=lambda f: int(''. join(filter(str. isdigit, f))))
#len(files)
for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        image = cv2.imread(filename)
        encoding = feature_extractor(image, return_tensors="pt")
        pixel_values = encoding.pixel_values.to(device)
        outputs = model(pixel_values=pixel_values)
        logits = outputs.logits.cpu()
        ### Run the following if image are chosen from the directory
        upsampled_logits = nn.functional.interpolate(
                        logits,
                        size=(image.shape[0],image.shape[1]),
                        mode="bilinear",
                        align_corners=False,
                                                )
        pred_seg = upsampled_logits.argmax(dim=1)[0]
        ### visualize the results
        color_seg = np.zeros((pred_seg.shape[0], pred_seg.shape[1], 3), dtype=np.uint8)
        palette = np.array(ade_palette())
        for label, color in enumerate(palette):
                color_seg[pred_seg == label, :] = color
        color_seg = color_seg[..., ::-1]  # convert to BGR

        img = np.array(image) * 0.5 + color_seg * 0.5  # plot the image with the segmentation map
        img = img.astype(np.uint8)
        name = './{}/'.format(pathOut_seg)+ 'seg'+ str(i) + '.jpg'
        cv2.imwrite(name, img)     # save frame as JPG file
#'''




In [4]:
pathOut_seg='./seg_images/'
images_to_video(pathOut_seg,5,'./semantic_segmented_cow.MOV')